In [74]:
# Librerias a utilizar
import pandas as pd
import requests
from colorama import Fore
from bs4 import BeautifulSoup
from selenium import webdriver
from functools import reduce
import lxml

# **Web Scraping**

### Utilizando la técnica de `Web Scraping`, recopilamos información sobre la población de diferentes países en el período de 2015 a 2022.

In [75]:
# Creamos una lista donde guardaremos los datos
lista = []
# Instanciamos el año inicial para empezar las iteraciones 
i = 2015

while i <= 2022:
    website = f"https://www.populationpyramid.net/population-size-per-country/{i}/"
    # Cargar la página con Selenium
    driver = webdriver.Chrome()
    driver.get(website)

    # Esperar a que la página se cargue por completo
    driver.implicitly_wait(10)

    # Capturar el contenido de la tabla con BeautifulSoup
    content = driver.page_source
    soup = BeautifulSoup(content, 'lxml')
    table = soup.find_all("table", {"class": "data-table"})

    driver.quit()

    rows = table[0].find_all('tr')

    rows = str(rows)

    soup = BeautifulSoup(rows, 'html.parser')

    paises = []

    for tr in soup.find_all('tr'):
        # Buscar las celdas de datos para cada país
        tds = tr.find_all('td')
        if len(tds) == 4:
            # Extraer la información de cada celda
            country = tds[2].text
            population = tds[3].text

            # Agregar la información a la lista de países
            paises.append({'country': country, f'population{i}': population})

    df = pd.DataFrame(paises)
    lista.append(df)

    i += 1

# Realizar el merge de todos los dataframes en la lista
df_web_scraping = reduce(lambda left, right: pd.merge(left, right, on='country', how='outer'), lista)
df_web_scraping.index +=1

In [76]:
# Revisamos el dataframe
df_web_scraping.head()

,country,population2015,population2016,population2017,population2018,population2019,population2020,population2021,population2022
1,China,"1,393,715,449","1,401,889,681","1,410,275,957","1,417,069,469","1,421,864,031","1,424,929,781","1,425,893,465","1,425,887,337"
2,India,"1,322,866,505","1,338,636,340","1,354,195,680","1,369,003,307","1,383,112,051","1,396,387,127","1,407,563,842","1,417,173,173"
3,United States of America,"324,607,777","327,210,198","329,791,231","332,140,038","334,319,671","335,942,004","336,997,624","338,289,857"
4,Indonesia,"259,091,970","261,850,182","264,498,853","267,066,843","269,582,878","271,857,971","273,753,191","275,501,339"
5,Pakistan,"210,969,298","213,524,840","216,379,655","219,731,480","223,293,280","227,196,741","231,402,117","235,824,863"


In [77]:
# Listamos las columnas del dataframe
data_top = list(df_web_scraping.columns.values)

# Se elimina la columnas de population ya que estas son a las que se les realizará el melt
i = 2015
while i <= 2022:
    data_top.remove(f"population{i}")
    i += 1
data_top    

['country']

In [78]:
# Se utiliza pandas.melt para reorganizar la estructura del dataframe
df_web_scraping = df_web_scraping.melt(data_top, var_name="Year", value_name="population")
df_web_scraping.head()

,country,Year,population
0,China,population2015,"1,393,715,449"
1,India,population2015,"1,322,866,505"
2,United States of America,population2015,"324,607,777"
3,Indonesia,population2015,"259,091,970"
4,Pakistan,population2015,"210,969,298"


In [79]:
# Mediante el str.slice extraemos solo los años
df_web_scraping["Year"] = df_web_scraping["Year"].str.slice(10)

In [80]:
# Comprobamos el resultado final
df_web_scraping[df_web_scraping["country"] == "China"]

,country,Year,population
0,China,2015,"1,393,715,449"
201,China,2016,"1,401,889,681"
402,China,2017,"1,410,275,957"
603,China,2018,"1,417,069,469"
804,China,2019,"1,421,864,031"
1005,China,2020,"1,424,929,781"
1206,China,2021,"1,425,893,465"
1407,China,2022,"1,425,887,337"


In [81]:
# Exportamos los datos a un archivo CSV para poder transformarlos posteriormente en SQL
df_web_scraping.to_csv("Datasets/Population (2015 - 2022).csv",sep = "|",index=False)

# **API**

### Obtenemos información sobre las características de diferentes países mediante una `API`

In [82]:
# Utilizando un ciclo while, se realiza una iteración para extraer la información necesaria de la API
i = 1
total_df = []
while i <= 6:
    url= (f"https://api.worldbank.org/v2/country?page={i}")
    response=requests.get(url,params={'format':'json'})
    # Extraemos los datos en formato JSON de la respuesta
    data=response.json()
    # Convertimos los datos en un dataframe de pandas normalizado
    df=pd.json_normalize(data[1])
    total_df.append(df)
    i+=1
# Finalmente unimos todos los dataframes de la lista total_df en uno solo    
df_api=pd.concat(total_df,ignore_index=True)  
df_api.index += 1

In [83]:
# Eliminamos las columnas que no son relevantes para el análisis que deseamos realizar
df_api.drop(["iso2Code","region.iso2code","adminregion.id","adminregion.iso2code","adminregion.value","incomeLevel.id",
             "incomeLevel.iso2code","lendingType.id","lendingType.value","lendingType.iso2code","region.id"],axis=1,inplace=True)

In [84]:
# Revisamos el dataframe
df_api.head()

,id,name,capitalCity,longitude,latitude,region.value,incomeLevel.value
1,ABW,Aruba,Oranjestad,-70.0167,12.5167,Latin America & Caribbean,High income
2,AFE,Africa Eastern and Southern,,,,Aggregates,Aggregates
3,AFG,Afghanistan,Kabul,69.1761,34.5228,South Asia,Low income
4,AFR,Africa,,,,Aggregates,Aggregates
5,AFW,Africa Western and Central,,,,Aggregates,Aggregates


In [85]:
# Comprobamos el resultado final
df_api[(df_api["name"] == "Finland")]

,id,name,capitalCity,longitude,latitude,region.value,incomeLevel.value
99,FIN,Finland,Helsinki,24.9525,60.1608,Europe & Central Asia,High income


In [86]:
# Exportamos los datos a un archivo CSV para poder transformarlos posteriormente en SQL
df_api.to_csv("Datasets/Income.csv",sep = "|",index=False) 

# **CSV**

### Realizamos la integración de archivos `CSV` para consolidar la información en un solo conjunto de datos

##### Normalizamos los dataframes para asegurarnos de que todos tengan la misma estructura, lo que nos permitirá realizar análisis y comparaciones consistentes 

In [87]:
df1 = pd.read_csv("Datasets/2015.csv",sep=",")
df1.drop(["Standard Error","Dystopia Residual","Region","Family","Happiness Rank"],axis=1 ,inplace=True)
df1["Year"] = 2015
df_2015 = df1
df_2015.head()

,Country,Happiness Score,Economy (GDP per Capita),Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Year
0,Switzerland,7.587,1.39651,0.94143,0.66557,0.41978,0.29678,2015
1,Iceland,7.561,1.30232,0.94784,0.62877,0.14145,0.43630,2015
2,Denmark,7.527,1.32548,0.87464,0.64938,0.48357,0.34139,2015
3,Norway,7.522,1.45900,0.88521,0.66973,0.36503,0.34699,2015
4,Canada,7.427,1.32629,0.90563,0.63297,0.32957,0.45811,2015


In [88]:
df2 = pd.read_csv("Datasets/2016.csv",sep=",")
df2.drop(["Lower Confidence Interval","Upper Confidence Interval","Dystopia Residual","Region","Family","Happiness Rank"],axis=1 ,inplace=True)
df2["Year"] = 2016
df_2016 = df2
df_2016.head()

,Country,Happiness Score,Economy (GDP per Capita),Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Year
0,Denmark,7.526,1.44178,0.79504,0.57941,0.44453,0.36171,2016
1,Switzerland,7.509,1.52733,0.86303,0.58557,0.41203,0.28083,2016
2,Iceland,7.501,1.42666,0.86733,0.56624,0.14975,0.47678,2016
3,Norway,7.498,1.57744,0.79579,0.59609,0.35776,0.37895,2016
4,Finland,7.413,1.40598,0.81091,0.57104,0.41004,0.25492,2016


In [89]:
df3 = pd.read_csv("Datasets/2017.csv",sep=",")
df3.drop(["Whisker.high","Whisker.low","Dystopia.Residual","Family","Happiness.Rank"],axis=1 ,inplace=True)
df3["Year"] = 2017
df_2017 = df3
df_2017.head()
df_2017.rename(columns={"Happiness.Score" :"Happiness Score" ,
                        "Economy..GDP.per.Capita." : "Economy (GDP per Capita)",
                          "Health..Life.Expectancy." : "Health (Life Expectancy)",
                            "Trust..Government.Corruption." : "Trust (Government Corruption)"  },inplace=True)

df_2017 = df_2017[["Country","Happiness Score","Economy (GDP per Capita)","Health (Life Expectancy)","Freedom","Trust (Government Corruption)","Generosity","Year"]].copy()

df_2017.head()

,Country,Happiness Score,Economy (GDP per Capita),Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Year
0,Norway,7.537,1.616463,0.796667,0.635423,0.315964,0.362012,2017
1,Denmark,7.522,1.482383,0.792566,0.626007,0.400770,0.355280,2017
2,Iceland,7.504,1.480633,0.833552,0.627163,0.153527,0.475540,2017
3,Switzerland,7.494,1.564980,0.858131,0.620071,0.367007,0.290549,2017
4,Finland,7.469,1.443572,0.809158,0.617951,0.382612,0.245483,2017


In [90]:
df4 = pd.read_csv("Datasets/2018.csv",sep=",")
df4.pop("Social support")
df4.pop("Overall rank")
df4["Year"] = 2018
df_2018 = df4

df_2018.rename(columns={"Happiness.Score" :"Happiness Score" ,
                        "Country or region" : "Country",
                        "Score" : "Happiness Score",
                        "GDP per capita" : "Economy (GDP per Capita)",
                          "Healthy life expectancy" : "Health (Life Expectancy)",
                            "Perceptions of corruption" : "Trust (Government Corruption)" , "Freedom to make life choices" : "Freedom"
                              },inplace=True)

df_2018 = df_2018[["Country","Happiness Score","Economy (GDP per Capita)","Health (Life Expectancy)","Freedom","Trust (Government Corruption)","Generosity","Year"]].copy()
df_2018.head()

,Country,Happiness Score,Economy (GDP per Capita),Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Year
0,Finland,7.632,1.305,0.874,0.681,0.393,0.202,2018
1,Norway,7.594,1.456,0.861,0.686,0.340,0.286,2018
2,Denmark,7.555,1.351,0.868,0.683,0.408,0.284,2018
3,Iceland,7.495,1.343,0.914,0.677,0.138,0.353,2018
4,Switzerland,7.487,1.420,0.927,0.660,0.357,0.256,2018


In [91]:
df5 = pd.read_csv("Datasets/2019.csv",sep=",")
df5.pop("Social support")
df5.pop("Overall rank")
df5["Year"] = 2019
df_2019 = df5
df_2019.rename(columns={ "Country or region" : "Country",
                        "Score" : "Happiness Score",
                        "GDP per capita" : "Economy (GDP per Capita)",
                          "Healthy life expectancy" : "Health (Life Expectancy)",
                            "Perceptions of corruption" : "Trust (Government Corruption)" , "Freedom to make life choices" : "Freedom"
                              },inplace=True)

df_2019 = df_2019[["Country","Happiness Score","Economy (GDP per Capita)","Health (Life Expectancy)","Freedom","Trust (Government Corruption)","Generosity","Year"]].copy()
df_2019.head()

,Country,Happiness Score,Economy (GDP per Capita),Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Year
0,Finland,7.769,1.340,0.986,0.596,0.393,0.153,2019
1,Denmark,7.600,1.383,0.996,0.592,0.410,0.252,2019
2,Norway,7.554,1.488,1.028,0.603,0.341,0.271,2019
3,Iceland,7.494,1.380,1.026,0.591,0.118,0.354,2019
4,Netherlands,7.488,1.396,0.999,0.557,0.298,0.322,2019


In [92]:
df6 = pd.read_csv("Datasets/2020.csv",sep=",")
df6.drop(["Standard error of ladder score","upperwhisker","lowerwhisker","Ladder score in Dystopia","Logged GDP per capita",
          "Healthy life expectancy","Explained by: Social support","Freedom to make life choices","Generosity","Dystopia + residual",
          "Regional indicator","Explained by: Perceptions of corruption","Social support"],axis=1,inplace=True)
df6["Year"] = 2020
df_2020 = df6
df_2020.rename(columns={"Ladder score" :"Happiness Score" ,
                        "Country name" : "Country",
                        "Explained by: Log GDP per capita" : "Economy (GDP per Capita)",
                          "Explained by: Healthy life expectancy" : "Health (Life Expectancy)",
                            "Perceptions of corruption" : "Trust (Government Corruption)" , "Explained by: Freedom to make life choices" : "Freedom",
                            "Explained by: Generosity" : "Generosity"
                              },inplace=True)

df_2020 = df_2020[["Country","Happiness Score","Economy (GDP per Capita)","Health (Life Expectancy)","Freedom","Trust (Government Corruption)","Generosity","Year"]].copy()
df_2020.head()

,Country,Happiness Score,Economy (GDP per Capita),Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Year
0,Finland,7.8087,1.285190,0.961271,0.662317,0.195445,0.159670,2020
1,Denmark,7.6456,1.326949,0.979333,0.665040,0.168489,0.242793,2020
2,Switzerland,7.5599,1.390774,1.040533,0.628954,0.303728,0.269056,2020
3,Iceland,7.5045,1.326502,1.000843,0.661981,0.711710,0.362330,2020
4,Norway,7.4880,1.424207,1.008072,0.670201,0.263218,0.287985,2020


In [93]:
df7 = pd.read_csv("Datasets/2021.csv",sep=",")
df7.drop(["Standard error of ladder score","upperwhisker","lowerwhisker","Ladder score in Dystopia","Logged GDP per capita",
          "Healthy life expectancy","Explained by: Social support","Freedom to make life choices","Generosity","Dystopia + residual",
          "Regional indicator","Explained by: Perceptions of corruption","Social support"],axis=1,inplace=True)
df7["Year"] = 2021
df_2021 = df7
df_2021.rename(columns={"Ladder score" :"Happiness Score" ,
                        "Country name" : "Country",
                        "Explained by: Log GDP per capita" : "Economy (GDP per Capita)",
                          "Explained by: Healthy life expectancy" : "Health (Life Expectancy)",
                            "Perceptions of corruption" : "Trust (Government Corruption)" , "Explained by: Freedom to make life choices" : "Freedom",
                            "Explained by: Generosity" : "Generosity"
                              },inplace=True)

df_2021 = df_2021[["Country","Happiness Score","Economy (GDP per Capita)","Health (Life Expectancy)","Freedom","Trust (Government Corruption)","Generosity","Year"]].copy()
df_2021.head()

,Country,Happiness Score,Economy (GDP per Capita),Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Year
0,Finland,7.842,1.446,0.741,0.691,0.186,0.124,2021
1,Denmark,7.620,1.502,0.763,0.686,0.179,0.208,2021
2,Switzerland,7.571,1.566,0.816,0.653,0.292,0.204,2021
3,Iceland,7.554,1.482,0.772,0.698,0.673,0.293,2021
4,Netherlands,7.464,1.501,0.753,0.647,0.338,0.302,2021


In [94]:
df8 = pd.read_csv("Datasets/2022.csv",sep=",",decimal=",")
df8.pop("Whisker-high")
df8.pop("Whisker-low")
df8.pop("Dystopia (1.83) + residual")
df8.pop("Explained by: Social support")
df8.pop("RANK")
df8["Year"] = 2022
df_2022 = df8
df_2022.rename(columns={"Happiness score" :"Happiness Score" ,
                        "Explained by: GDP per capita" : "Economy (GDP per Capita)",
                          "Explained by: Healthy life expectancy" : "Health (Life Expectancy)",
                            "Explained by: Perceptions of corruption" : "Trust (Government Corruption)" , "Explained by: Freedom to make life choices" : "Freedom",
                            "Explained by: Generosity" : "Generosity"
                              },inplace=True)

df_2022 = df_2022[["Country","Happiness Score","Economy (GDP per Capita)","Health (Life Expectancy)","Freedom","Trust (Government Corruption)","Generosity","Year"]].copy() 
df_2022.head()



,Country,Happiness Score,Economy (GDP per Capita),Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Year
0,Finland,7.821,1.892,0.775,0.736,0.534,0.109,2022
1,Denmark,7.636,1.953,0.777,0.719,0.532,0.188,2022
2,Iceland,7.557,1.936,0.803,0.718,0.191,0.270,2022
3,Switzerland,7.512,2.026,0.822,0.677,0.461,0.147,2022
4,Netherlands,7.415,1.945,0.787,0.651,0.419,0.271,2022


In [95]:
# Concatenamos todos los dataframes en uno solo
df_table = pd.concat([df_2015,df_2016,df_2017,df_2018,df_2019,df_2020,df_2021,df_2022],ignore_index=True)
df_table


,Country,Happiness Score,Economy (GDP per Capita),Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Year
0,Switzerland,7.587,1.39651,0.94143,0.66557,0.41978,0.29678,2015
1,Iceland,7.561,1.30232,0.94784,0.62877,0.14145,0.43630,2015
2,Denmark,7.527,1.32548,0.87464,0.64938,0.48357,0.34139,2015
3,Norway,7.522,1.45900,0.88521,0.66973,0.36503,0.34699,2015
4,Canada,7.427,1.32629,0.90563,0.63297,0.32957,0.45811,2015
...,...,...,...,...,...,...,...,...
1226,Rwanda*,3.268,0.78500,0.46200,0.62100,0.54400,0.18700,2022
1227,Zimbabwe,2.995,0.94700,0.27000,0.32900,0.10500,0.10600,2022
1228,Lebanon,2.955,1.39200,0.63100,0.10300,0.03400,0.08200,2022
1229,Afghanistan,2.404,0.75800,0.28900,0.00000,0.00500,0.08900,2022


In [96]:
# Comprobamos
df_table[(df_table["Country"]) == "Finland"]

,Country,Happiness Score,Economy (GDP per Capita),Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Year
5,Finland,7.4060,1.290250,0.889110,0.641690,0.413720,0.233510,2015
162,Finland,7.4130,1.405980,0.810910,0.571040,0.410040,0.254920,2016
319,Finland,7.4690,1.443572,0.809158,0.617951,0.382612,0.245483,2017
470,Finland,7.6320,1.305000,0.874000,0.681000,0.393000,0.202000,2018
626,Finland,7.7690,1.340000,0.986000,0.596000,0.393000,0.153000,2019
782,Finland,7.8087,1.285190,0.961271,0.662317,0.195445,0.159670,2020
935,Finland,7.8420,1.446000,0.741000,0.691000,0.186000,0.124000,2021
1084,Finland,7.8210,1.892000,0.775000,0.736000,0.534000,0.109000,2022


In [97]:
# Exportamos los datos a un archivo CSV para poder transformarlos posteriormente en SQL
df_table.to_csv("Datasets/Happiness (2015 - 2022).csv",sep = "|",index=False)